# MINI-PROJ-01

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1132793413819759975
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4857331712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17596140081709492338
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [5]:
tf.__version__

'2.9.1'

In [6]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [8]:
print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [9]:
print(X_test.shape)

(10000, 28, 28)


In [10]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='selu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=256, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [11]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []


for i in range(k):
    print('processing fold #', i)

    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    with tf.device("/device:GPU:0"):
        model.fit(X_train_part, y_train_part,
                  epochs=num_epochs, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/100
1688/1688 [==============================] - 42s 20ms/step - loss: 0.4639 - accuracy: 0.8386
Epoch 2/100
1688/1688 [==============================] - 34s 20ms/step - loss: 0.3145 - accuracy: 0.8907
Epoch 3/100
1688/1688 [==============================] - 34s 20ms/step - loss: 0.2736 - accuracy: 0.9027
Epoch 4/100
1688/1688 [==============================] - 34s 20ms/step - loss: 0.2431 - accuracy: 0.9139
Epoch 5/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.2197 - accuracy: 0.9229
Epoch 6/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1972 - accuracy: 0.9300
Epoch 7/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1780 - accuracy: 0.9382
Epoch 8/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1621 - accuracy: 0.9434
Epoch 9/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1442 - accuracy: 0.9487
Epoch 10/100
1688/1688 [===========

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0109 - accuracy: 0.9966
Epoch 79/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0095 - accuracy: 0.9968
Epoch 80/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0108 - accuracy: 0.9966
Epoch 81/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0094 - accuracy: 0.9970
Epoch 82/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0097 - accuracy: 0.9969
Epoch 83/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0079 - accuracy: 0.9972
Epoch 84/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0107 - accuracy: 0.9966
Epoch 85/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0086 - accuracy: 0.9972
Epoch 86/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0098 - accuracy: 0.9970
Epoch 87/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 35s 21ms/step - loss: 0.0136 - accuracy: 0.9957
Epoch 55/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0167 - accuracy: 0.9949
Epoch 56/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0140 - accuracy: 0.9956
Epoch 57/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0156 - accuracy: 0.9948
Epoch 58/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0143 - accuracy: 0.9952
Epoch 59/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0138 - accuracy: 0.9952
Epoch 60/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0138 - accuracy: 0.9953
Epoch 61/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0129 - accuracy: 0.9954
Epoch 62/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0118 - accuracy: 0.9960
Epoch 63/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0271 - accuracy: 0.9907
Epoch 31/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0267 - accuracy: 0.9911
Epoch 32/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0285 - accuracy: 0.9906
Epoch 33/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0271 - accuracy: 0.9907
Epoch 34/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0228 - accuracy: 0.9921
Epoch 35/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0222 - accuracy: 0.9926
Epoch 36/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0255 - accuracy: 0.9919
Epoch 37/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0239 - accuracy: 0.9919
Epoch 38/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0209 - accuracy: 0.9934
Epoch 39/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.1979 - accuracy: 0.9298
Epoch 7/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1778 - accuracy: 0.9373
Epoch 8/100
1688/1688 [==============================] - 36s 22ms/step - loss: 0.1592 - accuracy: 0.9442
Epoch 9/100
1688/1688 [==============================] - 36s 22ms/step - loss: 0.1424 - accuracy: 0.9494
Epoch 10/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1304 - accuracy: 0.9542
Epoch 11/100
1688/1688 [==============================] - 36s 22ms/step - loss: 0.1144 - accuracy: 0.9589
Epoch 12/100
1688/1688 [==============================] - 36s 22ms/step - loss: 0.1018 - accuracy: 0.9646
Epoch 13/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0953 - accuracy: 0.9671
Epoch 14/100
1688/1688 [==============================] - 36s 22ms/step - loss: 0.0885 - accuracy: 0.9691
Epoch 15/100
1688/1688 [==============================] - 36s 

...

1688/1688 [==============================] - 37s 22ms/step - loss: 0.0095 - accuracy: 0.9970
Epoch 89/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0090 - accuracy: 0.9972
Epoch 90/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0076 - accuracy: 0.9973
Epoch 91/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0103 - accuracy: 0.9969
Epoch 92/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0088 - accuracy: 0.9972
Epoch 93/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0078 - accuracy: 0.9975
Epoch 94/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0091 - accuracy: 0.9973
Epoch 95/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0093 - accuracy: 0.9972
Epoch 96/100
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0084 - accuracy: 0.9974
Epoch 97/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0110 - accuracy: 0.9964
Epoch 65/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0131 - accuracy: 0.9959
Epoch 66/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0123 - accuracy: 0.9961
Epoch 67/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0109 - accuracy: 0.9966
Epoch 68/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0122 - accuracy: 0.9961
Epoch 69/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0110 - accuracy: 0.9964
Epoch 70/100
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0096 - accuracy: 0.9970
Epoch 71/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0115 - accuracy: 0.9964
Epoch 72/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0099 - accuracy: 0.9969
Epoch 73/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0224 - accuracy: 0.9924
Epoch 41/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0217 - accuracy: 0.9929
Epoch 42/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0188 - accuracy: 0.9935
Epoch 43/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0205 - accuracy: 0.9929
Epoch 44/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0200 - accuracy: 0.9935
Epoch 45/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0172 - accuracy: 0.9944
Epoch 46/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0186 - accuracy: 0.9940
Epoch 47/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0169 - accuracy: 0.9941
Epoch 48/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0160 - accuracy: 0.9945
Epoch 49/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0721 - accuracy: 0.9751
Epoch 17/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0685 - accuracy: 0.9756
Epoch 18/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0621 - accuracy: 0.9784
Epoch 19/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0553 - accuracy: 0.9807
Epoch 20/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0539 - accuracy: 0.9817
Epoch 21/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0500 - accuracy: 0.9824
Epoch 22/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0447 - accuracy: 0.9846
Epoch 23/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0420 - accuracy: 0.9849
Epoch 24/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0389 - accuracy: 0.9871
Epoch 25/100
1688/1688 [==============================] - 3

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0082 - accuracy: 0.9969
Epoch 94/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0086 - accuracy: 0.9972
Epoch 95/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0071 - accuracy: 0.9979
Epoch 96/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0095 - accuracy: 0.9967
Epoch 97/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0072 - accuracy: 0.9976
Epoch 98/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0083 - accuracy: 0.9976
Epoch 99/100
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0087 - accuracy: 0.9971
Epoch 100/100
188/188 [==============================] - 2s 8ms/step - loss: 0.4530 - accuracy: 0.9313


In [12]:
all_scores

[0.9296666383743286,
 0.9351666569709778,
 0.9256666898727417,
 0.9330000281333923,
 0.9298333525657654,
 0.9298333525657654,
 0.9323333501815796,
 0.9348333477973938,
 0.9294999837875366,
 0.9313333630561829]

In [13]:
np.mean(all_scores)

0.9311166763305664

In [11]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='selu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [14]:
k = 10
num_val = len(X_train) // k
num_epochs = 35
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=1, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/35
1688/1688 [==============================] - 37s 21ms/step - loss: 0.4719 - accuracy: 0.8351
Epoch 2/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.3173 - accuracy: 0.8871
Epoch 3/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.2757 - accuracy: 0.9027
Epoch 4/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.2445 - accuracy: 0.9134
Epoch 5/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.2201 - accuracy: 0.9222
Epoch 6/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1974 - accuracy: 0.9317
Epoch 7/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1789 - accuracy: 0.9374
Epoch 8/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1622 - accuracy: 0.9428
Epoch 9/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1438 - accuracy: 0.9493
Epoch 10/35
1688/1688 [=====================

Epoch 7/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1770 - accuracy: 0.9378
Epoch 8/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1608 - accuracy: 0.9435
Epoch 9/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1428 - accuracy: 0.9498
Epoch 10/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1347 - accuracy: 0.9524
Epoch 11/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1186 - accuracy: 0.9593
Epoch 12/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.1052 - accuracy: 0.9631
Epoch 13/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0951 - accuracy: 0.9667
Epoch 14/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0896 - accuracy: 0.9689
Epoch 15/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0813 - accuracy: 0.9715
Epoch 16/35
1688/1688 [==============================] - 3

1688/1688 [==============================] - 35s 21ms/step - loss: 0.0985 - accuracy: 0.9655
Epoch 14/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0928 - accuracy: 0.9672
Epoch 15/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0815 - accuracy: 0.9713
Epoch 16/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0735 - accuracy: 0.9747
Epoch 17/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0665 - accuracy: 0.9764
Epoch 18/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0623 - accuracy: 0.9782
Epoch 19/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0573 - accuracy: 0.9800
Epoch 20/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0550 - accuracy: 0.9808
Epoch 21/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0477 - accuracy: 0.9840
Epoch 22/35
1688/1688 [==============================] - 35s 21ms/s

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0575 - accuracy: 0.9807
Epoch 20/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0497 - accuracy: 0.9829
Epoch 21/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0491 - accuracy: 0.9822
Epoch 22/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0456 - accuracy: 0.9845
Epoch 23/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0416 - accuracy: 0.9854
Epoch 24/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0401 - accuracy: 0.9864
Epoch 25/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0375 - accuracy: 0.9873
Epoch 26/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0360 - accuracy: 0.9879
Epoch 27/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0335 - accuracy: 0.9892
Epoch 28/35
1688/1688 [==============================] - 35s 21ms/s

1688/1688 [==============================] - 36s 21ms/step - loss: 0.0379 - accuracy: 0.9872
Epoch 26/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0352 - accuracy: 0.9879
Epoch 27/35
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0363 - accuracy: 0.9883
Epoch 28/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0334 - accuracy: 0.9883
Epoch 29/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0323 - accuracy: 0.9888
Epoch 30/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0296 - accuracy: 0.9902
Epoch 31/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0285 - accuracy: 0.9904
Epoch 32/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0294 - accuracy: 0.9904
Epoch 33/35
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0286 - accuracy: 0.9905
Epoch 34/35
1688/1688 [==============================] - 36s 21ms/s

In [21]:
all_scores

[0.9334999918937683,
 0.9244999885559082,
 0.925000011920929,
 0.9276666641235352,
 0.9350000023841858,
 0.9326666593551636,
 0.9358333349227905,
 0.9348333477973938,
 0.9316666722297668,
 0.934333324432373]

In [21]:
np.mean(all_scores)

0.9312499940395356

In [22]:
np.mean(all_scores)

0.9314999997615814